In [17]:
import pandas as pd
import numpy as np
import warnings

warnings.filterwarnings('ignore')
myData=pd.read_csv('imputedData.csv')
myData.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [18]:
#One hot encoding of categorical variables

#Create list with features to be dummified cols.
nonum_feats_names = ['Month','OperatingSystems','Browser','Region','VisitorType','Weekend']


#Boolean to dummify logic
#Weekend_map={False:0,True:1}
#pd.get_dummies(myData['Weekend'].map(Weekend_map).astype('category')


dataModel = pd.concat([myData[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']],
                       pd.get_dummies(myData[nonum_feats_names].astype('category')),myData['Revenue']],axis=1)
                      
dataModel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 57 columns):
Administrative                   12330 non-null int64
Administrative_Duration          12330 non-null float64
Informational                    12330 non-null int64
Informational_Duration           12330 non-null float64
ProductRelated                   12330 non-null int64
ProductRelated_Duration          12330 non-null float64
BounceRates                      12330 non-null float64
ExitRates                        12330 non-null float64
PageValues                       12330 non-null float64
TrafficType                      12330 non-null int64
SpecialDay                       12330 non-null float64
Month_Aug                        12330 non-null uint8
Month_Dec                        12330 non-null uint8
Month_Feb                        12330 non-null uint8
Month_Jul                        12330 non-null uint8
Month_June                       12330 non-null uint8
Month_Mar    

In [19]:
#Label Encoding of revenue

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
myData['Revenue'] = le.fit_transform(myData['Revenue'])
myData['Revenue'].value_counts()

myData['Revenue'].head()

0    0
1    0
2    0
3    0
4    0
Name: Revenue, dtype: int64

In [20]:
# getting dependent and independent variables

x = dataModel
# removing the target column revenue from x
x = x.drop(['Revenue'], axis = 1)

y = myData['Revenue']

# checking the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (12330, 56)
Shape of y: (12330,)


In [21]:
# splitting the data

from sklearn.model_selection import train_test_split

x_baseTrain, x_baseTest, y_baseTrain, y_baseTest = train_test_split(x, y, test_size = 0.3, random_state = 42)

# checking the shapes

print("Shape of x_train :", x_baseTrain.shape)
print("Shape of y_train :", y_baseTrain.shape)
print("Shape of x_test :", x_baseTest.shape)
print("Shape of y_test :", y_baseTest.shape)

Shape of x_train : (8631, 56)
Shape of y_train : (8631,)
Shape of x_test : (3699, 56)
Shape of y_test : (3699,)


In [22]:
#Scaling, Feature Selection not done yet.
#Correlation not yet considered.
# MODELLING ____ LOGISTIC REGRESSION

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
import seaborn as sns
import matplotlib.pyplot as plt

model = LogisticRegression()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)
#plt.rcParams['figure.figsize'] = (6, 6)
#sns.heatmap(cm ,annot = True)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))





Training Accuracy : 0.8839068474104971
Testing Accuracy : 0.881859962151933
ROC AUC Score : 0.6682469520681401
[[3056   68]
 [ 369  206]]
              precision    recall  f1-score   support

           0       0.89      0.98      0.93      3124
           1       0.75      0.36      0.49       575

    accuracy                           0.88      3699
   macro avg       0.82      0.67      0.71      3699
weighted avg       0.87      0.88      0.86      3699

{'fit_time': array([0.17054415, 0.09175396, 0.12466621, 0.12267137, 0.07978654,
       0.09574342, 0.10372186, 0.10870957, 0.14361548, 0.10272551]), 'score_time': array([0.00698185, 0.00698185, 0.00598431, 0.00598431, 0.00598407,
       0.00598431, 0.00598407, 0.00698161, 0.00698185, 0.00598359]), 'test_accuracy': array([0.88194444, 0.87152778, 0.88078704, 0.88760139, 0.89455388,
       0.88644264, 0.87485516, 0.89107764, 0.88167053, 0.88283063]), 'test_precision': array([0.75806452, 0.71698113, 0.73846154, 0.75714286, 0.8       

In [23]:
# MODELLING ____RANDOM FOREST

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

model = RandomForestClassifier()  #can try estimators=200,mex depth=30
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))


Training Accuracy : 0.9888773027459159
Testing Accuracy : 0.8913219789132197
ROC AUC Score : 0.7199671547068975
[[3026   98]
 [ 304  271]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      3124
           1       0.73      0.47      0.57       575

    accuracy                           0.89      3699
   macro avg       0.82      0.72      0.76      3699
weighted avg       0.88      0.89      0.88      3699

{'fit_time': array([0.08776569, 0.07479978, 0.07776427, 0.07679367, 0.07679439,
       0.07382894, 0.07577014, 0.07081008, 0.07679415, 0.07884479]), 'score_time': array([0.01398921, 0.0139637 , 0.01399016, 0.01393652, 0.01396322,
       0.01396275, 0.01296592, 0.01296592, 0.01396298, 0.01295114]), 'test_accuracy': array([0.89467593, 0.87615741, 0.90625   , 0.90961761, 0.89687138,
       0.90266512, 0.88644264, 0.89687138, 0.89559165, 0.88399072]), 'test_precision': array([0.74712644, 0.67088608, 0.8045977 , 0.81609195, 0.7340425

In [24]:
# MODELLING ____DECISION TREE

from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

model = DecisionTreeClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))


Training Accuracy : 1.0
Testing Accuracy : 0.8572587185725872
ROC AUC Score : 0.7359859154929578
[[2849  275]
 [ 253  322]]
              precision    recall  f1-score   support

           0       0.92      0.91      0.92      3124
           1       0.54      0.56      0.55       575

    accuracy                           0.86      3699
   macro avg       0.73      0.74      0.73      3699
weighted avg       0.86      0.86      0.86      3699

{'fit_time': array([0.05585027, 0.06482625, 0.05884194, 0.05884266, 0.05587697,
       0.05884242, 0.05684876, 0.05884194, 0.058846  , 0.05884218]), 'score_time': array([0.00698185, 0.00598454, 0.00598431, 0.00698113, 0.00598383,
       0.00698209, 0.00698018, 0.00698256, 0.00597978, 0.00598669]), 'test_accuracy': array([0.875     , 0.84490741, 0.85532407, 0.8783314 , 0.85863268,
       0.85979143, 0.86558517, 0.83429896, 0.86542923, 0.8549884 ]), 'test_precision': array([0.59285714, 0.5       , 0.53333333, 0.60294118, 0.53846154,
       0.543

In [25]:
# MODELLING ____NAIVE BAYES


from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

model = GaussianNB()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))



Training Accuracy : 0.7670026648128838
Testing Accuracy : 0.7742633144092999
ROC AUC Score : 0.7336781161275956
[[2476  648]
 [ 187  388]]
              precision    recall  f1-score   support

           0       0.93      0.79      0.86      3124
           1       0.37      0.67      0.48       575

    accuracy                           0.77      3699
   macro avg       0.65      0.73      0.67      3699
weighted avg       0.84      0.77      0.80      3699

{'fit_time': array([0.01495981, 0.01496077, 0.01495981, 0.01495934, 0.01495957,
       0.01695585, 0.01695395, 0.01495934, 0.01595783, 0.01396275]), 'score_time': array([0.00797844, 0.00797868, 0.00797939, 0.00797915, 0.00797844,
       0.00897598, 0.00797939, 0.00797868, 0.00797844, 0.00797844]), 'test_accuracy': array([0.76388889, 0.74305556, 0.78240741, 0.77056779, 0.78563152,
       0.77867903, 0.73812283, 0.74739282, 0.76566125, 0.75986079]), 'test_precision': array([0.33796296, 0.33206107, 0.37837838, 0.36514523, 0.3907563

In [26]:
# MODELLING ____KNN

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

model = KNeighborsClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))


# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.8979260804078323
Testing Accuracy : 0.8532035685320357
ROC AUC Score : 0.6150963090797751
[[3001  123]
 [ 420  155]]
              precision    recall  f1-score   support

           0       0.88      0.96      0.92      3124
           1       0.56      0.27      0.36       575

    accuracy                           0.85      3699
   macro avg       0.72      0.62      0.64      3699
weighted avg       0.83      0.85      0.83      3699

{'fit_time': array([0.04188728, 0.0369277 , 0.03989315, 0.03094411, 0.03593063,
       0.03590202, 0.03690124, 0.03593159, 0.03593111, 0.03590369]), 'score_time': array([0.24936485, 0.2403574 , 0.24337602, 0.22639394, 0.23337674,
       0.24035764, 0.23237896, 0.24035597, 0.2443459 , 0.23138142]), 'test_accuracy': array([0.86111111, 0.84606481, 0.86574074, 0.87137891, 0.87253766,
       0.86442642, 0.85515643, 0.86906141, 0.87703016, 0.87238979]), 'test_precision': array([0.62068966, 0.50847458, 0.65      , 0.671875  , 0.6666666

In [27]:
# MODELLING ____Linear SVC

from sklearn.svm import LinearSVC
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

model = LinearSVC()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.7521724018074383
Testing Accuracy : 0.7585834009191673
ROC AUC Score : 0.75561376162111
[[2374  750]
 [ 143  432]]
              precision    recall  f1-score   support

           0       0.94      0.76      0.84      3124
           1       0.37      0.75      0.49       575

    accuracy                           0.76      3699
   macro avg       0.65      0.76      0.67      3699
weighted avg       0.85      0.76      0.79      3699

{'fit_time': array([0.68317318, 0.65536523, 0.67899942, 0.66322565, 0.65524697,
       0.66023421, 0.66325355, 0.6582396 , 0.65824008, 0.66724253]), 'score_time': array([0.00698161, 0.        , 0.00698233, 0.00698185, 0.00698137,
       0.00598454, 0.0059557 , 0.00698137, 0.00598359, 0.00595713]), 'test_accuracy': array([0.52314815, 0.86458333, 0.82986111, 0.87485516, 0.85747393,
       0.8424102 , 0.85399768, 0.71147161, 0.86426914, 0.79118329]), 'test_precision': array([0.23371648, 0.69767442, 0.45255474, 0.60683761, 0.75      ,

In [28]:
# MODELLING ____Ada Boost Classifier

from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate

model = AdaBoostClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.8983895261267524
Testing Accuracy : 0.8888888888888888
ROC AUC Score : 0.750454823804487
[[2972  152]
 [ 259  316]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.94      3124
           1       0.68      0.55      0.61       575

    accuracy                           0.89      3699
   macro avg       0.80      0.75      0.77      3699
weighted avg       0.88      0.89      0.88      3699

{'fit_time': array([0.53856015, 0.4996357 , 0.51659203, 0.50063515, 0.49863935,
       0.49368143, 0.49864006, 0.49764323, 0.50063682, 0.49265599]), 'score_time': array([0.05587769, 0.05388331, 0.0568738 , 0.05886889, 0.05388284,
       0.05385351, 0.05288553, 0.0528841 , 0.05388045, 0.0538826 ]), 'test_accuracy': array([0.90046296, 0.87384259, 0.88657407, 0.90382387, 0.91772885,
       0.89455388, 0.8783314 , 0.88412514, 0.90023202, 0.88863109]), 'test_precision': array([0.73076923, 0.62626263, 0.66363636, 0.73148148, 0.76271186

In [29]:
# MODELLING ____XGBOOST classifier

from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate

model = XGBClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.9210983663538408
Testing Accuracy : 0.8991619356582861
ROC AUC Score : 0.7785317040583422
[[2979  145]
 [ 228  347]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3124
           1       0.71      0.60      0.65       575

    accuracy                           0.90      3699
   macro avg       0.82      0.78      0.80      3699
weighted avg       0.89      0.90      0.90      3699

{'fit_time': array([0.85870337, 0.82581878, 0.82282615, 0.83678865, 0.81983471,
       0.79886341, 0.79487371, 0.80085826, 0.77997565, 0.81316161]), 'score_time': array([0.01396322, 0.01795983, 0.01496029, 0.01396275, 0.01495957,
       0.01396322, 0.01396298, 0.01396275, 0.01561975, 0.01596069]), 'test_accuracy': array([0.91203704, 0.8900463 , 0.90972222, 0.92468134, 0.9212051 ,
       0.91425261, 0.89918888, 0.91193511, 0.90603248, 0.90487239]), 'test_precision': array([0.75892857, 0.68224299, 0.75      , 0.80357143, 0.7876106

In [30]:
# MODELLING ____gradient boosting Classifier

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score, cross_validate

model = GradientBoostingClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.9230680106592515
Testing Accuracy : 0.8948364422816978
ROC AUC Score : 0.7702947725880979
[[2971  153]
 [ 236  339]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3124
           1       0.69      0.59      0.64       575

    accuracy                           0.89      3699
   macro avg       0.81      0.77      0.79      3699
weighted avg       0.89      0.89      0.89      3699

{'fit_time': array([1.05914116, 1.04225183, 1.05915332, 1.0701499 , 1.06311822,
       1.06418085, 1.07812858, 1.03903604, 1.04836583, 1.08410001]), 'score_time': array([0.00997281, 0.01097155, 0.01097107, 0.01097035, 0.01097012,
       0.00998569, 0.01095772, 0.01562119, 0.01093674, 0.01097178]), 'test_accuracy': array([0.90856481, 0.88194444, 0.90625   , 0.9188876 , 0.91425261,
       0.90845886, 0.89571263, 0.90961761, 0.90603248, 0.89791183]), 'test_precision': array([0.74774775, 0.64545455, 0.74311927, 0.79439252, 0.7657657